<a href="https://colab.research.google.com/github/umbertogagl97/Tesi/blob/main/Impronte/AddestramentoScanner/Creazione_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Init**

## Import librerie

In [1]:
#Librerie
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, TensorDataset
import time
import os
import shutil
import copy
from PIL import Image

##Check device


In [ ]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

##Def variabili

In [2]:
#scanner
scanner_name = 'GreenBit'

#dataset
pathDataset=F'/content/gdrive/MyDrive/Dataset_impronte/training/{scanner_name}'

pathCrop=F'/content/gdrive/MyDrive/Dataset_impronte/training/{scanner_name}_patch/'

##Collegamento google drive

In [57]:
#collegamento google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#**Dataset**

##Creazione patch

In [58]:
def calc_size(n):
  return tuple(int(np.ceil(i * (80/100))) for i in n)

In [59]:
from PIL import ImageFile

Directory = os.listdir(pathDataset)
ImageFile.LOAD_TRUNCATED_IMAGES = True
for classe in Directory:
  classes_path = os.listdir(pathDataset+"/" + classe)  
  pathSave=pathCrop+classe
  print("working on: "+classe)
  for input in classes_path:
    #print('Working on '+ input)
    if (input.endswith('.png') or input.endswith('.bmp')):  
      img = Image.open(pathDataset+"/"+classe+'/' + input)
      n=img.size
      n_mod=calc_size(n)
      crop_transforms=transforms.TenCrop((n_mod[1],n_mod[0]))#.to(device)
      try: 
        crops=crop_transforms(img) #contiene le 10 patch
        i=0
        for crop in crops:
          i+=1
          split=input.split('.')
          name=split[0]+"_patch"+str(i)+"."+split[1]
          crop.save(pathCrop+classe+"/"+name)
      except: print(input,n,n_mod)    
  print("Finish: "+classe+". Salvate "+str(len(os.listdir(pathCrop + classe)))+" immagini")     
  

working on: Spoof
Finish: Spoof. Salvate 10000 immagini
working on: Live
Finish: Live. Salvate 10000 immagini


##Codice in caso di errore con il try catch, inserire singolarmente le immagini che danno errore

In [19]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
input="GreenBit_Live_021_2_4.png"
img = Image.open(pathDataset+"/Live/"+input)
n=img.size
n_mod=calc_size(n)
print(input,n,n_mod)
crop_transforms=transforms.TenCrop((n_mod[1],n_mod[0]))#.to(device)
try: 
  crops=crop_transforms(img) #contiene le 10 patch
  i=0
  for crop in crops:
    i+=1
    split=input.split('.')
    name=split[0]+"_patch"+str(i)+"."+split[1]
    print(save)
    crop.save(pathCrop+classe+"/"+name)
except: 
  print("except")
  print(input,n,n_mod)

GreenBit_Live_021_2_4.png (500, 500) (400, 400)


##check

In [60]:
dim_before=0
dim_after=0
for classe in ["Live","Spoof"]:
  dim_before+=(len(os.listdir(pathDataset+"/" + classe)))
  dim_after+=(len(os.listdir(pathCrop + classe)))
print(dim_before,dim_after)  
if dim_after==dim_before*10: print("OK")
else: print("BAD")

2000 20000
OK


controlla tutte le dimensioni di tutti i dataset

In [65]:
p="/content/gdrive/MyDrive/Dataset_impronte/training/"
print(len(os.listdir(p+"HiScan_patch"+"/Live")))
print(len(os.listdir(p+"HiScan_patch"+"/Spoof")))

print(len(os.listdir(p+"GreenBit_patch"+"/Live")))
print(len(os.listdir(p+"GreenBit_patch"+"/Spoof")))

print(len(os.listdir(p+"DigitalPersona_patch"+"/Live")))
print(len(os.listdir(p+"DigitalPersona_patch"+"/Spoof")))

print(len(os.listdir(p+"CrossMatch_patch"+"/Live")))
print(len(os.listdir(p+"CrossMatch_patch"+"/Spoof")))

6179
